### Load tensorflow

In [73]:
import tensorflow as tf

In [224]:
from sklearn.preprocessing import Normalizer

### Enable Eager Execution if you are using tensorflow 1.x

In [225]:
# Enable Eager Execution
tf.enable_eager_execution()

### Collect Data

In [226]:
import pandas as pd

In [227]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [228]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [229]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [230]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [231]:
data = data.drop(columns=['date','symbol'], axis = 1)

In [232]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [233]:
data = data.head(1000)

In [234]:
data.shape

(1000, 5)

In [235]:
data.sample(5)

,open,close,low,high,volume
521,36.380001,34.840000,34.470001,37.110001,2642400.0
455,29.150000,28.670000,28.549999,29.150000,13120900.0
14,110.419998,111.000000,107.300003,111.400002,563100.0
741,38.700001,38.020000,37.599998,38.700001,398900.0
532,48.270000,48.310001,47.950001,48.820000,497000.0


### Convert Float64 to Float32

In [236]:
data = data.astype('float32')


### Divide the data into train and test sets

In [237]:
from sklearn.model_selection import train_test_split

X1 = data.drop(columns= "close" , axis = 1) 
Y1 = data["close"]


### Normalize Train and Test Data 

In [238]:


train_x, test_x,train_y,test_y = train_test_split(X1, Y1, test_size=.3, random_state=7)

In [239]:
import numpy as np
transformer = Normalizer()
X1_train= transformer.fit_transform(train_x)
Y1_train=np.array(train_y)

## Building the graph in tensorflow

2.Define Weights and Bias

In [240]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [241]:
def prediction(x, w, b):
   
   xw_matmul = tf.matmul(x, w)
   y = tf.add(xw_matmul, b)
   
   return y

4.Loss (Cost) Function [Mean square error]

In [242]:
def loss(y_actual, y_predicted):
   
   diff = y_actual - y_predicted
   sqr = tf.square(diff)
   avg = tf.reduce_mean(sqr)
   
   return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [243]:
def train(x, y_actual, w, b, learning_rate=0.01):
   
   #Record mathematical operations on 'tape' to calculate loss
   with tf.GradientTape() as t:
       
       t.watch([w,b])
       
       current_prediction = prediction(x, w, b)
       current_loss = loss(y_actual, current_prediction)
   
   #Calculate Gradients for Loss with respect to Weights and Bias
   dw, db = t.gradient(current_loss,[w, b])
   
   #Update Weights and Bias
   w = w - learning_rate*dw
   b = b - learning_rate*db
   
   return w, b

## Execute the Graph for 100 epochs and observe the loss

In [244]:
for i in range(100):
   
   w, b = train(X1_train, Y1_train, w, b)
   print('Current Loss on iteration', i, loss(Y1_train, prediction(X1_train, w, b)).numpy())

Current Loss on iteration 0 8002.6514
Current Loss on iteration 1 7663.568
Current Loss on iteration 2 7351.0693
Current Loss on iteration 3 7063.065
Current Loss on iteration 4 6797.6504
Current Loss on iteration 5 6553.033
Current Loss on iteration 6 6327.599
Current Loss on iteration 7 6119.838
Current Loss on iteration 8 5928.3647
Current Loss on iteration 9 5751.9062
Current Loss on iteration 10 5589.279
Current Loss on iteration 11 5439.4014
Current Loss on iteration 12 5301.2764
Current Loss on iteration 13 5173.9775
Current Loss on iteration 14 5056.66
Current Loss on iteration 15 4948.547
Current Loss on iteration 16 4848.9023
Current Loss on iteration 17 4757.0684
Current Loss on iteration 18 4672.4365
Current Loss on iteration 19 4594.4424
Current Loss on iteration 20 4522.562
Current Loss on iteration 21 4456.313
Current Loss on iteration 22 4395.265
Current Loss on iteration 23 4338.999
Current Loss on iteration 24 4287.1455
Current Loss on iteration 25 4239.3525
Current L

### Get the shapes and values of W and b

In [245]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.5353245e-03]
 [2.5108384e-03]
 [2.5564781e-03]
 [3.3674854e+01]]
Bias:
 [33.67486]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [246]:
# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dense(1,  input_shape=(4,)))

In [247]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [248]:
model.fit(train_x,train_y, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 230us/sample - loss: nan         
Epoch 2/100
700/700 [==============================] - 0s 54us/sample - loss: nan
Epoch 3/100
700/700 [==============================] - 0s 50us/sample - loss: nan
Epoch 4/100
700/700 [==============================] - 0s 53us/sample - loss: nan
Epoch 5/100
700/700 [==============================] - 0s 63us/sample - loss: nan
Epoch 6/100
700/700 [==============================] - 0s 60us/sample - loss: nan
Epoch 7/100
700/700 [==============================] - 0s 57us/sample - loss: nan
Epoch 8/100
700/700 [==============================] - 0s 79us/sample - loss: nan
Epoch 9/100
700/700 [==============================] - 0s 56us/sample - loss: nan
Epoch 10/100
700/700 [==============================] - 0s 57us/sample - loss: nan
Epoch 11/100
700/700 [==============================] - 0s 43us/sample - loss: nan
Epoch 12/100
700/700 [==============================] - 0s 57us/sample - loss: nan
Epo

In [249]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [250]:
model.get_weights()

[array([[nan],
        [nan],
        [nan],
        [nan]], dtype=float32), array([nan], dtype=float32)]

In [222]:
from sklearn.model_selection import train_test_split


transformer = Normalizer()
X1= transformer.fit_transform(X1)
Y1= transformer.fit_transform(Y1)
train_x, test_x,train_y,test_y = train_test_split(X1, Y1, test_size=.3, random_state=7)

ValueError: Expected 2D array, got 1D array instead:
array=[125.839996 119.980003 114.949997 116.620003 114.970001 115.550003
 112.849998 114.379997 112.529999 110.379997 109.300003 110.
 111.949997 110.120003 111.       110.709999 112.580002 114.470001
 114.5      110.559998 114.050003 115.709999 114.019997 111.160004
 110.650002 107.519997 107.129997 107.839996 110.769997 111.239998
 111.599998 110.330002 113.040001 111.889999 111.559998 112.879997
 112.75     113.32     115.510002 116.779999 117.       117.190002
 116.949997 116.709999 116.489998 116.82     120.620003 120.629997
 120.699997 120.82     124.029999 121.269997 121.449997 121.470001
 119.379997 119.410004 118.720001 117.980003 118.739998 118.660004
 119.93     117.650002 114.07     115.470001 113.540001 113.410004
 114.639999 115.790001 117.989998 117.720001 118.449997 121.260002
 121.68     122.309998 123.010002 123.790001 123.300003 124.400002
 124.690002 124.150002 124.900002 125.959999 126.269997 126.559998
 127.6600

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [194]:
# iris= mount_drive("Iris-1.csv")
iris = pd.read_csv('Iris-2.csv')

In [195]:
iris.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

In [196]:
iris.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


In [197]:
iris["Species"] = pd.Categorical(iris["Species"]).codes

In [198]:
iris.drop("Id",axis=1,inplace=True)

### Splitting the data into feature set and target set

In [199]:
features = iris.drop(columns= ['Species']) 

target = iris[['Species']]

In [200]:

features.head(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [201]:
target.head(5)

,Species
0,0
1,0
2,0
3,0
4,0


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [202]:
from sklearn.preprocessing import LabelEncoder

In [203]:
Y_processed = pd.DataFrame(target).apply(LabelEncoder().fit_transform)

In [204]:
testY = tf.keras.utils.to_categorical(Y_processed, num_classes=3)

In [205]:
testY

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

### Divide the dataset into Training and test (70:30)

In [206]:
from sklearn.model_selection import train_test_split
transformer = Normalizer()
#features = transformer.fit_transform(features)

train_x, test_x,train_y,test_y = train_test_split(features, testY, test_size=.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [207]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [212]:
#train model
model.fit(train_x.values, train_y, validation_split=0.2, epochs=30)

Train on 84 samples, validate on 21 samples
Epoch 1/30
84/84 [==============================] - 1s 14ms/sample - loss: 1.0736 - acc: 0.2976 - val_loss: 1.2921 - val_acc: 0.3810
Epoch 2/30
84/84 [==============================] - 0s 262us/sample - loss: 1.0281 - acc: 0.2976 - val_loss: 1.1440 - val_acc: 0.3810
Epoch 3/30
84/84 [==============================] - 0s 262us/sample - loss: 0.9982 - acc: 0.2976 - val_loss: 1.0671 - val_acc: 0.3810
Epoch 4/30
84/84 [==============================] - 0s 297us/sample - loss: 0.9725 - acc: 0.3333 - val_loss: 1.0171 - val_acc: 0.3810
Epoch 5/30
84/84 [==============================] - 0s 202us/sample - loss: 0.9646 - acc: 0.4762 - val_loss: 0.9582 - val_acc: 0.3810
Epoch 6/30
84/84 [==============================] - 0s 309us/sample - loss: 0.9379 - acc: 0.5476 - val_loss: 0.9221 - val_acc: 0.3810
Epoch 7/30
84/84 [==============================] - 0s 202us/sample - loss: 0.9164 - acc: 0.6667 - val_loss: 0.8937 - val_acc: 0.3810
Epoch 8/30
84/84 [=

### Report Accuracy of the predicted values

In [213]:
model.evaluate(test_x,test_y)

45/45 [==============================] - 0s 8ms/sample - loss: 0.7459 - acc: 0.6444


[0.7459042800797356, 0.64444447]

In [214]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_7 (Batch multiple                  16        
_________________________________________________________________
dense_19 (Dense)             multiple                  50        
_________________________________________________________________
dense_20 (Dense)             multiple                  88        
_________________________________________________________________
dense_21 (Dense)             multiple                  27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________
